# mpt-7b-Instruct を SageMaker で Hosting
このノートブックは、mpt-7b を Instruction tuning した mpt-7b-Instruct を、ローカルで推論し、それを SageMaker で Hosting するノートブックです。  
モデルの詳細については [Hugging Face mpt-7b-instruct](https://huggingface.co/mosaicml/mpt-7b-instruct) や [MosaicML の blog](https://www.mosaicml.com/blog/mpt-7b) を参照ください。
一度ローカルで推論する都合上、ml.g5.2xlarge インスタンスを使用します。  
SageMaker Notebooks の `conda_pytorch_p39` カーネルと、SageMaker Studio Notebook の `PyTorch 1.13 Python 3.9 GPU Optimized` カーネルで動いた実績があります。

## ローカル推論
SageMaker で動かす前にローカルで動作確認を行う。
### ローカルで動かすためのライブラリをインストール
必要なモジュールをインストールする

In [ ]:
pip install transformers==4.26 einops sagemaker -U

### モジュール読み込み

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc

### モデルのダウンロード
tokenizer と model をダウンロードします。
[How to use](https://huggingface.co/mosaicml/mpt-7b-instruct#how-to-use) に沿って実行します。

In [ ]:
%%time

tokenizer = AutoTokenizer.from_pretrained(
    "mosaicml/mpt-7b-Instruct"
)

以下のセルはモデルを DL して読み込むため 8-9 分ほど時間がかかります。

In [ ]:
%%time
model = AutoModelForCausalLM.from_pretrained(
    "mosaicml/mpt-7b-Instruct", 
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda:0")

### モデルの保存
ローカルで推論する前に、モデルをストレージに出力して、再度読み込みます。  
SageMaker で Hosting する際はモデルをファイルから読み込むことが一般的で、ローカルで動かすときもその方法に則って行うと、SageMaker に移植しやすいためにこの手順を入れています。

In [ ]:
!rm -rf './model'
!mkdir -p './model/code'
model_dir = './model'
tokenizer.save_pretrained(model_dir)
model.save_pretrained(model_dir)

メモリ解放をします(OOM 対策)

In [ ]:
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

### モデルの再ロード
ファイルからモデルをロードします。
7 分程度かかります。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [ ]:
%%time

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).to("cuda:0")

### 推論する
prompt の形式は以下にすると良い結果が得られやすいです。  
(Instruction Tuning する際のトレーニングデータの形式に則った方式）
```text
{状況や前提}
### Instruction:
{命令文}
### Response:
```

In [ ]:
prompt = '''Python で フィボナッチ数列の 10 番目を知りたいです。
### Instruction:
フィボナッチ数列を求める関数とその関数の実行コードを記載してください。
### Response:'''

In [ ]:
%%time
inputs = tokenizer(prompt, return_tensors='pt').to("cuda:0")

In [ ]:
%%time
input_length = inputs.input_ids.shape[1]
with torch.no_grad():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=128, 
        do_sample=True, 
        temperature=0.01, 
        top_p=0.7, 
        top_k=50, 
        return_dict_in_generate=True
    )


In [ ]:
# 結果を出力する
token = outputs.sequences[0, input_length:]
output_str = tokenizer.decode(token)
output_str = output_str[:output_str.find('<|endoftext|>')]

print(output_str)

出力結果を以下のセルに貼り付けて、コードが実行できるか確認しましょう。

In [ ]:
# 出力を以下に貼り付けて実行
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

print(fib(10))

無事動いたらローカルでやっていたことを SageMaker の Hosting を使って再現します。

## SageMaker による推論

### モジュールのロードと定数の設定

In [ ]:
import sagemaker
import boto3
sm = boto3.client('sagemaker')
role = sagemaker.get_execution_role()

### 推論コードの作成
先程実行したコードをもとに記述していきます。  
まずは必要なモジュールを記述した requirements.txt を用意します。  
今回は [deep-learning-containers](https://github.com/aws/deep-learning-containers)の HuggingFace のコンテナを使います。  
einops だけ不足しているので requirements.txt に記載します。

In [ ]:
%%writefile model/code/requirements.txt
einops

先程実行したコードを SageMaker Inference 向けに改変します。
1. `model_fn` でモデルを読み込みます。先程は huggingface のモデルを直接ロードしましたが、`model_dir` に展開されたモデルを読み込みます。
2. `input_fn` で前処理を行います。
    * json 形式のみを受け付け他の形式は弾くようにします。
    * json 文字列を dict 形式に変換して `return` します。
3. `predict_fn` で推論します。`temperature` などのパラメータも合わせて入力します。
4. `output_fn` で json 形式にして `return` します。

In [ ]:
%%writefile model/code/inference.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

def model_fn(model_dir):
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        trust_remote_code=True,
        torch_dtype=torch.float16,
    ).to("cuda:0")
    return {'tokenizer':tokenizer,'model':model}

def input_fn(data, content_type):
    if content_type == 'application/json':
        data = json.loads(data)
    else:
        raise TypeError('content_type is only allowed application/json')
    return data

def predict_fn(data, model):
    inputs = model['tokenizer'](data['prompt'], return_tensors='pt').to("cuda:0")
    input_length = inputs.input_ids.shape[1]
    max_new_tokens = data['max_new_tokens']
    do_sample = data['do_sample']
    temperature = data['temperature']
    top_p = data['top_p']
    top_k = data['top_k']
    return_dict_in_generate = data['return_dict_in_generate']
    with torch.no_grad():
        outputs = model['model'].generate(
            **inputs, 
            max_new_tokens=max_new_tokens, 
            do_sample=do_sample, 
            temperature=temperature, 
            top_p=top_p, 
            top_k=top_k, 
            return_dict_in_generate=return_dict_in_generate
        )
    
    token = outputs.sequences[0, input_length:]
    output_str = model['tokenizer'].decode(token)
    
    return output_str

def output_fn(data, accept_type):
    if accept_type == 'application/json':
        data = json.dumps({'result' : data})
    else:
        raise TypeError('content_type is only allowed application/json')
    return data

### モデルアーティファクトの作成と S3 アップロード
アーティファクト(推論コード + モデル)を tar.gz に固めます。時間がかかるので `pigz` で並列処理を行います。  
ml.g5.2xlarge で 3-4 分ほどかかります。

※ SageMaker Studio を使っている場合は pigz が入っていないので、以下セルのコメントを解除してインストールしてください。

In [ ]:
# !apt update -y
# !apt install pigz -y

In [ ]:
%%time

!rm model.tar.gz
%cd model/
!tar  cv ./ | pigz -p 8 > ../model.tar.gz # 8 並列でアーカイブ
%cd ..

S3 にアップロードします。

In [ ]:
%%time

model_s3_uri = sagemaker.session.Session().upload_data(
    'model.tar.gz',
    key_prefix='mpt-7b-Instruct'
)
print(model_s3_uri)

### SageMaker SDK を用いてデプロイ

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
region = boto3.session.Session().region_name

In [ ]:
# 名前の設定
model_name = 'mpt-7b-Instruct'
endpoint_config_name = model_name + 'Config'
endpoint_name = model_name + 'Endpoint'

In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework='huggingface',
    region=region,
    version='4.26',
    image_scope='inference',
    base_framework_version='pytorch1.13',
    instance_type = 'ml.g5.xlarge'
)

In [ ]:
huggingface_model = HuggingFaceModel(
    model_data = model_s3_uri,
    role = role,
    image_uri = image_uri
)

In [ ]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

### SageMaker SDK で推論
model_fn の実行に時間がかかってしまい、エンドポイントが IN_SERVICE になっても、初回推論はしばらく動かないことがあります。  
CloudWatch Logs に以下のような表示がある場合はしばらく待てば使えるようになります。  
`[WARN] pool-3-thread-1 com.amazonaws.ml.mms.metrics.MetricCollector - worker pid is not available yet.`  
だいたい 6 分くらいかかるため、リトライを入れています。

In [ ]:
# prompt 確認
print(prompt)

In [ ]:
from time import sleep
request = {
    'prompt' : prompt,
    'max_new_tokens' : 128,
    'do_sample' : True,
    'temperature' : 0.01,
    'top_p' : 0.7,
    'top_k' : 50,
    'return_dict_in_generate' : True
}

for i in range(10):
    try:
        output_str = predictor.predict(request)['result']
        break
    except:
        sleep(60)

print(output_str[:output_str.find('<|endoftext|>')])

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## boto3 でデプロイと推論
標準だと SageMaker SDK が入っていない環境からデプロイや推論する場合(例:AWS Lambda など)は、boto3 でデプロイや推論することも多いです。  
以下のセルは boto3 で実行する方法を記述しています。
各 API の詳細は Document を確認してください。
[SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html)  
[SageMakerRuntime](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html)  

In [ ]:
import json
sm = boto3.client('sagemaker')
smr = boto3.client('sagemaker-runtime')
endpoint_inservice_waiter = sm.get_waiter('endpoint_in_service')

モデルの作成

In [ ]:
response = sm.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': image_uri,
        'ModelDataUrl': model_s3_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
            'SAGEMAKER_REGION': region,
        }
    },
    ExecutionRoleArn=role,
)

エンドポイントコンフィグの作成

In [ ]:
response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.g5.2xlarge',
        },
    ]
)

エンドポイントの作成

In [ ]:
response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

エンドポイント作成の完了を待つ

In [ ]:
endpoint_inservice_waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={'Delay': 5,}
)

推論を行います。  
ただし初回推論時のみモデルのロードに 7 分ほどかかるため、先程同様リトライを入れています。

In [ ]:
# prompt 確認
print(request)

In [ ]:
%%time

# 推論
smr = boto3.client('sagemaker-runtime')

for i in range(10):
    try:
        response = smr.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Accept='application/json',
            Body=json.dumps(request)
        )
        break
    except:
        sleep(60)
output_str = json.loads(response['Body'].read().decode('utf-8'))['result']
print(output_str[:output_str.find('<|endoftext|>')])

In [ ]:
response = smr.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Accept='application/json',
        Body=json.dumps(request)
)

お片付け

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_model(ModelName=model_name)